In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import cv2
from concurrent.futures import ThreadPoolExecutor

In [45]:
INPATH = Path('train/')
OUTPATH = Path('../aug/train/')
seg = pd.read_csv('train.csv')
seg2 = pd.read_csv('../aug/augment.csv', header=None)
test = pd.read_csv('sample_submission.csv')

In [47]:
len([f for f in OUTPATH.iterdir()])

68094

In [32]:
seg2.columns = seg.columns

In [33]:
seg2.head()

,Id,Target
0,d75fa7ac-93ea-434d-aa09-f56af3f3238d,25
1,3601c9cc-4d47-4b25-ad33-bb85ad5c8dd7,25
2,278cb8fa-b84e-4022-81c0-c8371ffa33f4,6
3,e08b8709-829c-44bc-828f-dfdbff3a9264,6
4,9149e078-8305-45f7-ba21-41d0d753de5e,7 21 25


In [27]:
len(seg),len(seg2)

(31072, 68093)

In [40]:
train_names = seg.Id.values
test_names = test.Id.values

In [48]:
def combine_rgb(fname, path=INPATH, out_path=OUTPATH, sz=256):
    '''open RGBA image from 4 different 1-channel files.
    return: numpy image with 3 channels'''
    flags = cv2.IMREAD_GRAYSCALE
    red = cv2.imread(str(path/(fname+ '_red.png')), flags)
    blue = cv2.imread(str(path/(fname+ '_blue.png')), flags)
    green = cv2.imread(str(path/(fname+ '_green.png')), flags)
    
    im = np.stack([red, green, blue], axis=-1)
#     im = cv2.resize(im, (sz, sz), interpolation = cv2.INTER_CUBIC)
    cv2.imwrite(str(out_path/(fname+'.png')), im)
    return rgba

In [49]:
with ThreadPoolExecutor(8) as e: e.map(combine_rgb, train_names)

In [42]:
with ThreadPoolExecutor(8) as e: e.map(combine_rgb, test_names)